In [ ]:
from os.path import join as opj
import os
import pandas as pd
from tqdm.auto import tqdm
from traffic.data import airports, aircraft
from traffic.core import Traffic, Flight
import plotly.express as px
import plotly.graph_objects as go

import scipy.stats as st
from datetime import timedelta

from scipy.signal import argrelextrema
import numpy as np

In [ ]:
width = 1000
height = width / 1.618
(width, height)

# Data loading


In [ ]:
df_landings = pd.read_parquet(opj("data", "osn23_landings_merged.parquet.gzip"))
print(f"{len(df_landings)} landings are in the dataset")
df_landings

### Filtering out general aviaiton / helicopters ...


In [ ]:
valid_types = ["L2J", "L4J", "L2T", "L3J", "L4T", "L1T", "L3T", "L1J"]
df_landings = df_landings.query("icaoaircrafttype in @valid_types")
l_invalid = ("FCK", "GAF", "PPU", "CFL", "ECK", "CALIBRAT", "PHLAB", "CALIBRA", "ENF")
df_landings = df_landings.query("not callsign.str.startswith(@l_invalid)")
df_landings = df_landings.query(
    "n_attempts<4"
)  # if there are more than 4 attempts it is most likely a trainning flight
print(
    f"Among {len(df_landings)} commercial aviation landings, {len(df_landings.query('GoA'))} are GoA"
)

In [ ]:
# check the percentage of rows that have a C40_BEARING value
print(
    f"{len(df_landings.query('C40_BEARING.notnull()'))/len(df_landings):.2%} of the landings have a C40_BEARING value"
)  # make the count per airport
for airport, group in df_landings.groupby("airport"):
    print(f"{airport}: {len(group.query('C40_BEARING.notnull()'))/len(group):.2%}")

### Where are the landings?


In [ ]:
fig = px.sunburst(
    df_landings.groupby(["airport"])
    .agg({"flight_id": "count", "country": "first"})
    .reset_index(),
    path=["country", "airport"],
    values="flight_id",
    labels={"country": "Country"},
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.data[0].textinfo = "label+percent entry+value"
fig.show()
fig.write_image("figures/landings_country_airport.pdf")

# Analysis


## High Level


### GoA Rate per market segment


In [ ]:
# Define the aggregation functions
agg_functions = {"GoA": ["mean", "sum"], "flight_id": "count"}

# Aggregate the data by market segment
df_agg = df_landings.groupby("market_segment").agg(agg_functions)

# Rename the columns
df_agg.columns = ["ga_rate", "n_ga", "n_landings"]

# Compute the Clopper-Pearson interval for the GoA rate
alpha = 0.05
n = df_agg["n_landings"]
k = df_agg["n_ga"]
lower_ci, upper_ci = st.beta.interval(1 - alpha, k, n - k + 1)
df_agg["ga_rate_lower"] = lower_ci * 1000
df_agg["ga_rate_upper"] = upper_ci * 1000
df_agg["ga_rate"] = df_agg["ga_rate"] * 1000

# Compute the lower and upper bounds of the confidence interval
df_agg["ga_rate_lb"] = df_agg["ga_rate"] - df_agg["ga_rate_lower"]
df_agg["ga_rate_ub"] = df_agg["ga_rate_upper"] - df_agg["ga_rate"]

# Filter out market segments with large confidence intervals
df_agg = df_agg.query("ga_rate_ub < 1")

# Create a scatter plot of the GoA rate per market segment
fig = px.scatter(
    df_agg.sort_values("ga_rate"),
    y="ga_rate",
    error_y="ga_rate_ub",
    error_y_minus="ga_rate_lb",
    color="n_landings",
    size=[1] * len(df_agg),
    size_max=8,
)
fig.update_layout(
    yaxis_title="GoA rate [/1000 landings]",
    xaxis_title="Market Segment",
    width=width,
    height=height,
    coloraxis_colorbar=dict(title="landings #"),
)
fig.show()
fig.write_image("figures/goa_rate_per_segment.pdf")

### GoA rate per typecode


In [ ]:
# Define the aggregation functions
agg_functions = {"GoA": ["mean", "sum"], "flight_id": "count"}

# Copy the landing data and add a new column for the typecode
df_temp = df_landings.copy()
df_temp["typecode"] = df_temp["typecode"].str[:3] + "x"

# Aggregate the data by typecode
df_agg = df_temp.groupby("typecode").agg(agg_functions)

# Rename the columns
df_agg.columns = ["ga_rate", "n_ga", "n_landings"]

# Compute the Clopper-Pearson interval for the GoA rate
alpha = 0.05
n = df_agg["n_landings"]
k = df_agg["n_ga"]
lower_ci, upper_ci = st.beta.interval(1 - alpha, k, n - k + 1)
df_agg["ga_rate_lower"] = lower_ci * 1000
df_agg["ga_rate_upper"] = upper_ci * 1000
df_agg["ga_rate"] = df_agg["ga_rate"] * 1000

# Compute the lower and upper bounds of the confidence interval
df_agg["ga_rate_lb"] = df_agg["ga_rate"] - df_agg["ga_rate_lower"]
df_agg["ga_rate_ub"] = df_agg["ga_rate_upper"] - df_agg["ga_rate"]

# Filter out typecodes with too large confidence intervals
df_agg = df_agg.query("(0 < ga_rate_ub < 1) or (0 < ga_rate_lb < 1)")

# Create a scatter plot of the GoA rate per typecode
fig = px.scatter(
    df_agg.sort_values("ga_rate"),
    y="ga_rate",
    error_y="ga_rate_ub",
    error_y_minus="ga_rate_lb",
    color="n_landings",
    size=[1] * len(df_agg),
    size_max=8,
)
fig.update_layout(
    yaxis_title="GoA rate [/1000 landings]",
    xaxis_title="Typecode",
    width=width,
    height=height,
    coloraxis_colorbar=dict(title="landings #"),
)
fig.write_image("figures/goa_rate_per_typecode.pdf")
fig.show()

### GoA rate per airport


In [ ]:
# Define the aggregation functions
agg_functions = {"GoA": ["mean", "sum"], "flight_id": "count"}

# Aggregate the data by airport
df_agg = df_landings.groupby("airport").agg(agg_functions)

# Rename the columns
df_agg.columns = ["ga_rate", "n_ga", "n_landings"]

# Compute the Clopper-Pearson interval for the GoA rate
alpha = 0.05
n = df_agg["n_landings"]
k = df_agg["n_ga"]
lower_ci, upper_ci = st.beta.interval(1 - alpha, k, n - k + 1)
df_agg["ga_rate_lower"] = lower_ci * 1000
df_agg["ga_rate_upper"] = upper_ci * 1000
df_agg["ga_rate"] = df_agg["ga_rate"] * 1000

# Compute the lower and upper bounds of the confidence interval
df_agg["ga_rate_lb"] = df_agg["ga_rate"] - df_agg["ga_rate_lower"]
df_agg["ga_rate_ub"] = df_agg["ga_rate_upper"] - df_agg["ga_rate"]

# Filter out airports with large confidence intervals
df_agg = df_agg.query("ga_rate_ub < 0.5")

# Create a scatter plot of the GoA rate per airport
fig = px.scatter(
    df_agg.sort_values("ga_rate"),
    y="ga_rate",
    error_y="ga_rate_ub",
    error_y_minus="ga_rate_lb",
    color="n_landings",
    size=[1] * len(df_agg),
    size_max=8,
)
fig.update_layout(
    yaxis_title="GoA rate",
    xaxis_title="Airport",
    width=width,
    height=height,
    coloraxis_colorbar=dict(title="Landings #"),
)
# add a line to represent the average GoA rate and label it
fig.add_hline(
    y=df_agg["ga_rate"].mean(),
    line_dash="dash",
    annotation_text="mean GoA rate",
    annotation_position="top right",
    annotation_font_size=15,
)
fig.show()
fig.write_image("figures/goa_rate_per_airport.pdf")

# Define the airport order dictionary
airport_order = {"airports": df_agg.sort_values("ga_rate").index.tolist()}

In [ ]:
df_agg.describe()

## Trajectory level


### Loading GoA Trajectories


In [ ]:
t_gas = Traffic.from_file(opj("data", f"goas19-23.parquet"))
t_gas = t_gas[df_landings.query("GoA").flight_id.unique()]
t_gas

In [ ]:
df_goas = df_landings.query("GoA")
list_goa_portions = []
for f in tqdm(t_gas):
    t0, tf = df_goas.query("flight_id==@f.flight_id")["attempt_times"].iloc[0][:2]
    t0 = pd.to_datetime(t0, utc=True)
    tf = pd.to_datetime(tf, utc=True)
    list_goa_portions.append(f.between(t0, tf))
goa_portions = Traffic.from_flights(list_goa_portions)
goa_portions

### Calculating GoA distance and fuel consumption


In [ ]:
def add_goa_metrics(f):
    try:
        # first we check the distance between the two landing attempts
        f.data = f.data.assign(d_start_end=f.distance())
        # we compute the length of the goa portioin
        f = f.cumulative_distance()
        # we compute the fuel consumption of the goa portion assuming 60T of initial mass (valid for a320s)
        f = f.fuelflow(initial_mass=60000)
    except:
        return
    return f


goa_portions = (
    goa_portions.iterate_lazy()
    .pipe(add_goa_metrics)
    .eval(desc="computing distance", max_workers=1)
)

In [ ]:
# Monkey patching the Flight class
def d_start_end(self):
    return self.data.d_start_end.iloc[0]


Flight.d_start_end = property(d_start_end)

### Agreagating all data in one


In [ ]:
infoGoA = (
    goa_portions.summary(
        [
            "flight_id",
            "callsign",
            "start",
            "stop",
            "destination",
            "cumdist_max",
            "d_start_end",
            "fuel_max",
            "typecode",
        ]
    )
    .eval()
    .sort_values("start")
    .rename(columns={"start": "first", "stop": "second"})
)
infoGoA["ga_time"] = infoGoA["second"] - infoGoA["first"]
infoGoA["ga_time_s"] = infoGoA["ga_time"].dt.total_seconds()
infoGoA["ga_time_m"] = infoGoA["ga_time"] / pd.Timedelta("60s")
infoGoA["year"] = infoGoA["first"].dt.year

# Filter summary
infoGoA

In [ ]:
infoGoA["speed"] = infoGoA.cumdist_max / infoGoA.ga_time_m * 60

### Filtering GoAs


In [ ]:
infoGoA_filtered = infoGoA.query("ga_time_s>200 & (d_start_end<2) & speed<300")
infoGoA_filtered.sort_values("ga_time_s", ascending=True)

In [ ]:
infoGoA_filtered.sort_values("cumdist_max", ascending=True)

In [ ]:
# # make 3 subplots (3 columns) and plot line_mapbox in each
# fig =

px.scatter_mapbox(
    t_gas[
        infoGoA_filtered.sort_values("cumdist_max", ascending=True)
        .tail(1)
        .flight_id.unique()
    ]
    .query("geoaltitude<11000")
    .data,
    lat="latitude",
    lon="longitude",
    hover_data=["flight_id", "timestamp"],
    mapbox_style="carto-positron",
    color="altitude",
    height=500,
    width=800,
)

### GoA Distances


In [ ]:
# sort box plot by cumdist median
var = "cumdist_max"
category_orders = {
    "destination": infoGoA_filtered.groupby("destination")[[var]]
    .median()
    .sort_values(var)
    .index
}
fig = px.box(
    infoGoA_filtered,
    x="destination",
    y=var,
    color="destination",
    points="outliers",
    category_orders=category_orders,
)
fig.update_yaxes(tickformat=".0f")
fig.update_layout(
    height=height,
    width=width,
    yaxis_type="log",
    xaxis_title="Airport",
    yaxis_title="GoA distance [NM]",
    showlegend=False,
)

fig.write_image("figures/goa_flown_distance.pdf")
fig.show()

In [ ]:
def percentile(n):
    def percentile_(x):
        return x.quantile(n)

    percentile_.__name__ = "percentile_{:02.0f}".format(n * 100)
    return percentile_


statistic = (
    infoGoA_filtered.groupby("destination")
    .agg(
        {
            "cumdist_max": [
                "min",
                percentile(0.25),
                "mean",
                "median",
                percentile(0.85),
                "max",
                "count",
            ]
        }
    )
    .sort_values(("cumdist_max", "median"))
)
# round values to 1 decimal
statistic = statistic.round(1)
statistic

### GoA Times


In [ ]:
# sort box plot by cumdist median
var = "ga_time_m"
category_orders = {
    "destination": infoGoA_filtered.groupby("destination")[[var]]
    .median()
    .sort_values(var)
    .index
}
fig = px.box(
    infoGoA_filtered,
    x="destination",
    y=var,
    color="destination",
    points="outliers",
    category_orders=category_orders,
)
fig.update_yaxes(tickformat=".0f")
fig.update_layout(
    height=height,
    width=width,
    yaxis_type="log",
    xaxis_title="Airport",
    yaxis_title="GoA Duration [min]",
    showlegend=False,
)
fig.write_image("figures/goa_time_s.pdf")
fig.show()

In [ ]:
statistic = (
    infoGoA_filtered.groupby("destination")
    .agg(
        {
            "ga_time_m": [
                "min",
                percentile(0.25),
                "mean",
                "median",
                percentile(0.85),
                "max",
                "count",
            ]
        }
    )
    .sort_values(("ga_time_m", "median"))
)
# round values to 1 decimal
statistic = statistic.round(1)
statistic

In [ ]:
statistic = (
    infoGoA_filtered.groupby("destination")
    .agg(
        {
            "cumdist_max": [percentile(0.25), "median", percentile(0.85), "max"],
            "ga_time_m": [percentile(0.25), "median", percentile(0.85), "max"],
        }
    )
    .sort_values(("ga_time_m", "median"))
)
# round values to 1 decimal
statistic = statistic.round(1)
statistic

### Fuel Consumption


In [ ]:
var = "fuel_max"
category_orders = {
    "destination": infoGoA_filtered.query('typecode=="A320"')
    .groupby("destination")[[var]]
    .median()
    .sort_values(var)
    .index
}
fig = px.box(
    infoGoA_filtered.query('typecode=="A320"'),
    x="destination",
    y=var,
    color="destination",
    points="outliers",
    category_orders=category_orders,
)
fig.update_yaxes(tickformat=".0f")
fig.update_layout(
    height=height,
    width=width,
    yaxis_type="log",
    xaxis_title="Airport",
    yaxis_title="GoA Fuel Consumption [Kg]",
    showlegend=False,
)
# save fig pdf
fig.write_image("figures/a320_goa_fuel.pdf")
fig.show()

In [ ]:
infoGoA_filtered

In [ ]:
infoGoA_filtered.query('typecode=="A320"').flight_id.unique()

In [ ]:
# objective, find when the aircraft initialise the go around
def get_a320_portions(f):
    try:
        first, second, *_ = f.aligned_on_ils(f.destination, angle_tolerance=0.15)
        # # find where the altitude is minimum
        idx = first.data.geoaltitude.idxmin()
        t0, d0 = first.data.loc[idx, ["timestamp", "distance"]]
        tf = second.query(f"distance>{d0}").data.timestamp.iloc[-1]
        # portion = f.between(t0, tf)
        portion = f.after(t0)
    except:
        return
    return portion.fuelflow(initial_mass=60000)


a320_portions = (
    t_gas[infoGoA_filtered.query('typecode=="A320"').flight_id.unique()]
    .iterate_lazy()
    .pipe(get_a320_portions)
    .eval(desc="computing fuel", max_workers=1)
)
a320_portions

In [ ]:
infoA320 = (
    a320_portions.summary(
        [
            "flight_id",
            "callsign",
            "start",
            "stop",
            "destination",
            "fuel_max",
            "typecode",
        ]
    )
    .eval()
    .sort_values("start")
)
infoA320

In [ ]:
fig = px.box(
    infoA320,
    x="destination",
    y="fuel_max",
    color="destination",
    category_orders={
        "destination": infoA320.groupby("destination")[["fuel_max"]]
        .median()
        .sort_values("fuel_max")
        .index
    },
)
fig.update_layout(
    height=height,
    width=width,
    yaxis_type="log",
    xaxis_title="Airport",
    yaxis_title="GoA Fuel Consumption [Kg]",
    showlegend=False,
)
fig.write_image("figures/a320_goa_fuel.pdf")
fig.show()

## Impact on other landing traffic


### Finding ASMA Sector


In [ ]:
def angular_difference(b1, b2):
    return 180 - np.abs(np.abs(b1 - b2) - 180)


def add_asma_sector(group):
    # Fit KDE to bearing data
    kde = st.gaussian_kde(group.C40_BEARING, bw_method=0.08)
    # Find all local maxima of the KDE
    maxima_indices = argrelextrema(kde.pdf(np.linspace(0, 360, 360 + 1)), np.greater)

    # Get the x values corresponding to the maxima indices
    modes = np.linspace(0, 360, 360 + 1)[maxima_indices]

    # sort the modes
    modes = np.sort(modes)

    # associate each data point of group.C40_BEARING to the closest mode
    # compute the distance between each data point and each mode
    distances = np.abs(np.subtract.outer(group.C40_BEARING.values, modes))
    # associate each data point to the closest mode
    group["ASMA_sector"] = np.argmin(distances, axis=1)

    # if there is lest than 15 deg difference between the first and last mode, merge them on the mean
    if angular_difference(modes[0], modes[-1]) < 15:
        group.loc[group["ASMA_sector"] == len(modes) - 1, "ASMA_sector"] = 0
    group["ASMA_sector"] = group["ASMA_sector"].astype(str)
    return group


groups = []
for (airport, runway), group in tqdm(df_landings.groupby(["airport", "AP_C_RWY"])):
    if len(group) < 1000:
        continue
    group = group.dropna(subset=["C40_BEARING"])
    group = add_asma_sector(group)
    groups.append(group)

df_landings_with_asma = pd.concat(groups)

In [ ]:
import plotly.graph_objects as go

# Filter data
selected_airport = "EDDF"
selected_runway = "25L"
subset = (
    df_landings_with_asma.query(
        "airport==@selected_airport & AP_C_RWY==@selected_runway"
    )
    .sample(10000)
    .sort_values("ASMA_sector")
)

fig1 = px.scatter_mapbox(
    subset,
    lat="C40_CROSS_LAT",
    lon="C40_CROSS_LON",
    color="ASMA_sector",
    mapbox_style="carto-positron",
    zoom=8,
    height=800,
    width=800,
)
# update map center to the airport
fig1.update_layout(
    mapbox_center={
        "lat": airports[selected_airport].centroid.coords[0][1],
        "lon": airports[selected_airport].centroid.coords[0][0],
    },
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

fig1.update_layout(showlegend=False)

#
fig3 = px.histogram(
    subset, x="C40_BEARING", color="ASMA_sector", height=500, width=500, nbins=100
)
kde = st.gaussian_kde(subset.C40_BEARING, bw_method=0.08)
x = np.linspace(0, 360, 1000)
fig3.add_trace(
    go.Scatter(
        x=x,
        y=kde.pdf(x)
        * np.histogram(subset.C40_BEARING, bins=100)[0].max()
        / np.max(kde.pdf(x)),
        name="kde",
        mode="lines",
        line=dict(color="black", dash="dash", width=0.5),
    )
)
fig3.update_layout(
    xaxis_title="ASMA entry bearing [deg]",
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
    ),
)
fig1.show()
fig3.show()
fig1.write_image("figures/asma_map2sector.pdf")
fig3.write_image("figures/asma_bearing2sector.pdf")

In [ ]:
tqdm.pandas()
import numpy as np

# Define the columns to group by
group_cols = ["airport", "ASMA_sector", "AP_C_RWY", "AC_CLASS"]

# Calculate the time difference between stop time and C40 cross time
df_landings_with_asma["time40"] = (
    df_landings_with_asma["landing_time"] - df_landings_with_asma["C40_CROSS_TIME"]
)

# Calculate the 20th percentile of time40 for each group of columns where GoA is False
dict_ref_times = (
    df_landings_with_asma.query("not GoA")
    .groupby(group_cols)
    .progress_apply(lambda x: x["time40"].quantile(q=0.2))
    .to_dict()
)

df_landings_with_asma["ref_time"] = df_landings_with_asma.progress_apply(
    lambda x: dict_ref_times.get(
        (x.airport, x.ASMA_sector, x.AP_C_RWY, x.AC_CLASS), np.nan
    ),
    axis=1,
)
# # Drop rows with missing values in the group columns
# df_landings_with_asma = df_landings_with_asma.dropna(subset=group_cols)

# # Calculate the extra time for each row by subtracting the reference time for the group
df_landings_with_asma["extra_time"] = (
    df_landings_with_asma["time40"] - df_landings_with_asma["ref_time"]
)

# # Convert the extra time to seconds
df_landings_with_asma["extra_time_s"] = df_landings_with_asma[
    "extra_time"
].dt.total_seconds()

#### Effect of GoA on ASMA time

In [ ]:
from collections import defaultdict

airport2windows = {}

# Create a new dictionary for the airport in the 'w_results' dictionary
# airport2windows = {}
for airport, airport_group in df_landings_with_asma.groupby("airport"):
    if airport in airport2windows:
        continue
    res_windows = defaultdict(list)

    # Retrieve the times of first GoA Attempt
    time_gas = airport_group.query("GoA").attempt_times.apply(lambda x: x[0])
    window_minutes = 10
    for t_goa in pd.to_datetime(time_gas, utc=True):
        # Compute the reference time as the median extra time in the 20 minutes before the GoA
        t_before = t_goa - timedelta(minutes=20)
        subset_before = airport_group.query("@t_before < landing_time < @t_goa")

        # Skip the current GoA if there is a GoA 20min before or an hour after
        thour = t_goa + timedelta(hours=1)
        subset_next_hour = airport_group.query(f"@t_goa < landing_time < @thour")
        if (
            len(subset_next_hour.query("GoA")) > 1
            or len(subset_before.query("GoA")) > 0
        ):
            continue

        reference_time = subset_before.extra_time_s.median()
        for w0 in range(-20, 51):
            # Extract all landings happening dm minutes before and after the GoA
            tw0 = t_goa + timedelta(minutes=w0)
            twf = tw0 + timedelta(minutes=(window_minutes))
            subset_after = airport_group.query(
                "(@tw0 < landing_time < @twf) and (not GoA)"
            )
            additional_time = subset_after.extra_time_s.median() - reference_time
            if np.isnan(additional_time):
                continue
            res_windows[w0].append(additional_time)
    airport2windows[airport] = {
        k: np.nanmedian(v) for k, v in res_windows.items() if len(v) > 0
    }
    print(airport, airport2windows[airport])

We do the same when there is no GoA. We create randomly 10 different control groups for each airport:

In [ ]:
from collections import defaultdict

airport2windows_control = defaultdict(list)
for airport, airport_group in df_landings_with_asma.groupby("airport"):

    # Retrieve the times of first GoA Attempt
    for _ in range(10):
        # we randomly chose 
        time_lndgs = airport_group.query("not GoA").landing_time.sample(len(airport_group.query("GoA")))
        window_minutes = 10
        res_windows = defaultdict(list)
        for t_lnd in pd.to_datetime(time_lndgs, utc=True):
            # Compute the reference time as the median extra time in the 20 minutes before the GoA
            t_before = t_lnd - timedelta(minutes=20)
            subset_before = airport_group.query("@t_before < landing_time < @t_goa")

            # Skip the current GoA if there is a GoA 20min before or an hour after
            thour = t_lnd + timedelta(hours=1)
            subset_next_hour = airport_group.query(f"@t_goa < landing_time < @thour")
            if (
                len(subset_next_hour.query("GoA")) > 1
                or len(subset_before.query("GoA")) > 0
            ):
                continue

            reference_time = subset_before.extra_time_s.median()
            for w0 in range(-20, 51):
                # Extract all landings happening dm minutes before and after the GoA
                tw0 = t_lnd + timedelta(minutes=w0)
                twf = tw0 + timedelta(minutes=(window_minutes))
                subset_after = airport_group.query(
                    "(@tw0 < landing_time < @twf) and (not GoA)"
                )
                additional_time = subset_after.extra_time_s.median() - reference_time
                if np.isnan(additional_time):
                    continue
                res_windows[w0].append(additional_time)

        airport2windows_control[airport].append(
            {k: np.nanmedian(v) for k, v in res_windows.items() if len(v) > 0}
        )


In [ ]:
from plotly.subplots import make_subplots

df = pd.DataFrame.from_dict(airport2windows, orient="index")
df = df[sorted(df.columns)]
df = df.iloc[:, 5:] # we only start from [-15 min ;-5 min]
df.columns = [f"[{c} ; {c+10}]" for c in df.columns]

fig = make_subplots(
    cols=2, rows=10, shared_xaxes=True, vertical_spacing=0.01, horizontal_spacing=0.01
)

for i, (airport, row) in enumerate(df.iterrows()):
    plotcol = i % 2 + 1
    plotrow = i // 2 + 1
    annotation_y = i + 1
    if plotcol == 2:
        plotrow += 1
        annotation_y = i + 2

    for n in range(len(airport2windows_without2[airport])):
        without = pd.DataFrame.from_dict(
            airport2windows_control[airport][n], orient="index"
        )
        without = without[sorted(without.columns)]
        without = without.loc[-15:]
        fig.add_trace(
            go.Scatter(
                x=row.index,
                y=without.values.flatten(),
                name="Control group: no GoA",
                mode="lines",
                line=dict(color="blue"),
                opacity=0.2,
                showlegend=(i<1) & (n<1),
            ),
            col=plotcol,
            row=plotrow,
            
        )

    fig.add_trace(
        go.Scatter(
            x=row.index,
            y=row.values,
            name="GoA",
            mode="lines",
            line=dict(color="red"),
            showlegend=(i<1),
        ),
        col=plotcol,
        row=plotrow,
    )

    fig.add_annotation(
        xref="paper",
        yref=f"y{annotation_y}",
        x=plotcol / 2 - 0.05,
        y=50,
        text=airport,
        showarrow=False,
    )
    fig.update_yaxes(tickvals=[0, 100, 200], row=plotrow, col=plotcol)

fig.update_yaxes(range=[df.values.min(), df.values.max()])
fig.update_layout(
    height=29.7 / 2.54 * 96,
    width=21 / 2.54 * 96,
    # showlegend=False,
    margin=dict(t=0, l=100, r=0, b=100),
)

# only display y axis ticks on the first column
for i in range(10):
    fig.update_yaxes(tickvals=[0, 50, 100], ticktext=["", "", ""], row=i + 1, col=2)
    fig.update_yaxes(tickvals=[0, 50, 100], row=i + 1, col=1)


# using anotation add y title to the left of the first column
fig.add_annotation(
    xref="x domain",
    yref="paper",
    x=-0.1,
    y=0.5,
    xanchor="right",
    text="Median ASMA additional time [s]",
    showarrow=False,
    font=dict(size=15),
    textangle=-90,
)

fig.add_annotation(
    xref="paper",
    yref="paper",
    x=0.5,
    y=-0.075,
    yanchor="top",
    text="Time window before / after GoA [min]",
    showarrow=False,
    font=dict(size=15),
)



for i in range(10):
    fig.add_vline(
        x="[-9 ; 1]",
        line_dash="dash",
        row=i + 1,
        col=1,
    )
    if i > 0:
        fig.add_vline(
            x="[-9 ; 1]",
            line_dash="dash",
            row=i + 1,
            col=2,
        )

for i in range(2):
    fig.update_xaxes(
        tickvals=[
            "[-15 ; -5]",
            "[-10 ; 0]",
             "[-5 ; 5]",
            "[0 ; 10]",
            "[5 ; 15]",
            "[10 ; 20]",
            "[15 ; 25]",
            "[20 ; 30]",
            "[25 ; 35]",
            "[30 ; 40]",
            "[35 ; 45]",
            "[40 ; 50]",
            "[45 ; 55]",
        ],
        row=10,
        col=i + 1,
    )

# change legend position to top right corner in plot
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="right",
        x=1,
    ),
)
fig.show()
fig.write_html("figures/extra_time.html")
fig.write_image("figures/extra_time.pdf")